# Packages and data loading

In [59]:
pip freeze

anyio @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/croot-t_zs64wy/anyio_1644482593257/work/dist
appdirs==1.4.4
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1649077682618/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/croot-wbf5edig/argon2-cffi-bindings_1644845754377/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1670263926556/work
async-generator==1.10
attrs==23.1.0
Babel @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_00k1rl2pus/croot/babel_1671781944131/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beautifulsoup4==4.12.2
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
blinker==1.6.3


In [60]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pathlib import Path  

In [61]:
# We load both network and stripe fees files, and concat them

stripe = pd.read_csv('Oct_stripe_RADAR.csv')



In [62]:
stripe['balance_transaction_description']

0         Card Account Updater (CAU) (2023-10-01): Updates
1                              Network Tokens (2023-10-01)
2           Card payments (2023-10-01): Stripe volume fees
3           Card payments (2023-10-01): Stripe volume fees
4           Card payments (2023-10-01): Stripe volume fees
                               ...                        
71637    Connect (2023-10-01 - 2023-10-31): Account Ini...
71638                       3D Secure (2023-10-31): Lookup
71639    Connect (2023-10-01 - 2023-10-31): Account Vol...
71640        Connect (2023-10-01 - 2023-10-31): Payout Fee
71641    Connect (2023-10-01 - 2023-10-31): Active Acco...
Name: balance_transaction_description, Length: 71642, dtype: object

In [63]:
# Then we merge both dataframe to work and perform analysis on an unique one

stripe['balance_transaction_reporting_category'] = stripe['balance_transaction_reporting_category'].replace('fee','stripe_fees')

df = stripe.copy()

# Some cleaning for future easier computation

df.card_brand = df['card_brand'].fillna(value='unknown')
df.card_funding = df['card_brand'].fillna(value='unknown')
df['month_year'] = pd.to_datetime(df['activity_at']).dt.strftime('%m-%Y')

df.head()

,balance_transaction_id,balance_transaction_created_at,balance_transaction_reporting_category,balance_transaction_component,event_type,activity_at,activity_interval_type,activity_start_date,activity_end_date,currency,...,card_country,statement_descriptor,payment_metadata[destination],refund_id,dispute_id,transfer_id,connected_account_id,connected_account_name,connected_account_country,connected_account_direct_charge_id
0,txn_1Nwc8mFq1BiPgN2KSczsMnBS,2023-10-02 02:45:48,stripe_fees,other_fee,fee,2023-10-01 23:59:59,range,2023-10-01,2023-10-01,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,txn_1Nwc95Fq1BiPgN2K2ZokO7dm,2023-10-02 02:46:07,stripe_fees,other_fee,NaN,2023-10-01 23:59:59,range,2023-10-01,2023-10-01,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,txn_1NwetxFq1BiPgN2KtZz09OOU,2023-10-02 05:42:41,stripe_fees,volume_fee,charge_captured,2023-10-01 19:19:58,instant,2023-10-01,2023-10-01,eur,...,FR,Puissante,NaN,NaN,NaN,NaN,acct_1JnNxk2EjHgjyRYx,Puissante,FR,NaN
3,txn_1NwetxFq1BiPgN2KtZz09OOU,2023-10-02 05:42:41,stripe_fees,volume_fee,charge_captured,2023-10-01 20:04:27,instant,2023-10-01,2023-10-01,eur,...,FR,Zelys,NaN,NaN,NaN,NaN,acct_1LT4a22HsmZAUOLF,Zelys,FR,NaN
4,txn_1NwetxFq1BiPgN2KtZz09OOU,2023-10-02 05:42:41,stripe_fees,volume_fee,charge_captured,2023-10-01 18:35:11,instant,2023-10-01,2023-10-01,eur,...,FR,Lilly Skin,NaN,NaN,NaN,NaN,acct_1M0AtQCMnGdwijAc,Lilly Skin,FR,NaN


# First insigths

In [64]:
# We display our dataframe columns name

df.columns

Index(['balance_transaction_id', 'balance_transaction_created_at',
       'balance_transaction_reporting_category',
       'balance_transaction_component', 'event_type', 'activity_at',
       'activity_interval_type', 'activity_start_date', 'activity_end_date',
       'currency', 'amount', 'customer_facing_currency',
       'customer_facing_amount', 'balance_transaction_description', 'fee_id',
       'automatic_payout_id', 'automatic_payout_effective_at', 'charge_id',
       'payment_intent_id', 'invoice_id', 'invoice_number', 'subscription_id',
       'payment_method_type', 'is_link', 'card_brand', 'card_funding',
       'card_country', 'statement_descriptor', 'payment_metadata[destination]',
       'refund_id', 'dispute_id', 'transfer_id', 'connected_account_id',
       'connected_account_name', 'connected_account_country',
       'connected_account_direct_charge_id'],
      dtype='object')

In [65]:
# We compute unique value count for each column and load it in a dataframe so we can save it as csv for later analysis

resultat = pd.DataFrame({'Nom de la colonne': df.nunique().index, 'Nombre d\'éléments uniques': df.nunique().values})

resultat


,Nom de la colonne,Nombre d'éléments uniques
0,balance_transaction_id,191
1,balance_transaction_created_at,189
2,balance_transaction_reporting_category,1
3,balance_transaction_component,3
4,event_type,9
5,activity_at,67515
6,activity_interval_type,2
7,activity_start_date,31
8,activity_end_date,31
9,currency,1


# Overall dive-deep 

In [66]:
# First we check which type of combination between fee categories and their name exist and their count

df[['balance_transaction_reporting_category','balance_transaction_component','balance_transaction_id']].groupby(['balance_transaction_reporting_category','balance_transaction_component']).size().reset_index(name='nb_transaction')


,balance_transaction_reporting_category,balance_transaction_component,nb_transaction
0,stripe_fees,other_fee,127
1,stripe_fees,per_auth_fee,47677
2,stripe_fees,volume_fee,23838


In [67]:
# Then we check which card brand could be involved in the transactions, and which type of card it is (debit/credit)

df[['card_brand','card_funding']].groupby(['card_brand','card_funding']).size().reset_index(name='nb_transaction')


,card_brand,card_funding,nb_transaction
0,American Express,credit,1594
1,Diners Club,credit,4
2,Discover,credit,6
3,MasterCard,credit,4904
4,MasterCard,debit,18074
5,MasterCard,prepaid,486
6,UnionPay,debit,8
7,Visa,credit,12013
8,Visa,debit,34352
9,Visa,prepaid,74


In [68]:
# Then we can combine both and display count of each card brand/funding combination and their corresponding fee nale/category

df[['card_brand','card_funding','balance_transaction_reporting_category','balance_transaction_component','balance_transaction_id']].groupby(['card_brand','card_funding','balance_transaction_reporting_category','balance_transaction_component']).count().rename(columns={'balance_transaction_id': 'nb'})



nb
card_brand       card_funding balance_transaction_reporting_category balance_transaction_component       
American Express credit       stripe_fees                            per_auth_fee                    1034
                                                                     volume_fee                       560
Diners Club      credit       stripe_fees                            per_auth_fee                       4
Discover         credit       stripe_fees                            per_auth_fee                       4
                                                                     volume_fee                         2
MasterCard       credit       stripe_fees                            per_auth_fee                    3293
                                                                     volume_fee                      1611
                 debit        stripe_fees                            per_auth_fee                   12069
                                                                     volume_fee                      6005
                 prepaid      stripe_fees                            per_auth_fee                     347
                                                                     volume_fee                       139
UnionPay         debit        stripe_fees                            per_auth_fee                       8
Visa             credit       stripe_fees                            per_auth_fee                    7973
                                                                     volume_fee                      4040
                 debit        stripe_fees                            per_auth_fee                   22891
                                                                     volume_fee                     11461
                 prepaid      stripe_fees                            per_auth_fee                      54
                                                                     volume_fee                        20

In [69]:
# Finally we compute fixed and variables mean fees for each combination

df[['card_brand','card_funding','balance_transaction_reporting_category','balance_transaction_component','balance_transaction_description','amount']].groupby(['card_brand','card_funding','balance_transaction_reporting_category','balance_transaction_component','balance_transaction_description'])[['amount']].mean()

amount
card_brand       card_funding balance_transaction_reporting_category balance_transaction_component balance_transaction_description                             
American Express credit       stripe_fees                            per_auth_fee                  Card payments (2023-10-01 - 2023-10-31): adjust...  0.050000
                                                                                                   Card payments (2023-10-01): Stripe per-authoriz... -0.170000
                                                                                                   Card payments (2023-10-02): Stripe per-authoriz... -0.127353
                                                                                                   Card payments (2023-10-03): Stripe per-authoriz... -0.120000
                                                                                                   Card payments (2023-10-04): Stripe per-authoriz... -0.120000
...                                                                                                                                                         ...
Visa             prepaid      stripe_fees                            volume_fee                    Card payments (2023-10-23): Stripe volume fees      0.882300
                                                                                                   Card payments (2023-10-24): Stripe volume fees     -0.100500
                                                                                                   Card payments (2023-10-26): Stripe volume fees     -0.186000
                                                                                                   Card payments (2023-10-29): Stripe volume fees     -0.472500
                                                                                                   Card payments (2023-10-30): Stripe volume fees     -1.457400

[423 rows x 1 columns]

In [80]:
# Finally we group by the type of event and save it as csv

# to_dl = df[['card_brand','card_funding','balance_transaction_reporting_category','balance_transaction_component','balance_transaction_description','amount']].groupby(['card_brand','card_funding','balance_transaction_reporting_category','balance_transaction_component','balance_transaction_description'])[['amount']].sum()

A = df[df['balance_transaction_description'].str.contains('Payout', case=False)]
B = df[df['balance_transaction_description'].str.contains('3D', case=False)]
C = df[df['balance_transaction_description'].str.contains('Radar', case=False)]

tot = pd.concat([A,B,C], axis=0)

to_dl = tot[['balance_transaction_description','amount']].groupby('balance_transaction_description')[['amount']].sum()

to_dl.to_csv('stripe_p&l_RADAR.csv')

------------

# Deeper analysis

To get more insights and precision for this analysis, one may focus on the different features/events happening druing a given transaction. 

This part will allow having a better understanding on how the fees are applied, occuring and which value they have.

In [79]:

filtered_df = df[df['balance_transaction_description'].str.contains('Payout', case=False)]
filtered_df.card_funding

71640    unknown
Name: card_funding, dtype: object

In [78]:
df[df['balance_transaction_description'].str.contains('3D', case=False)].head()

,balance_transaction_id,balance_transaction_created_at,balance_transaction_reporting_category,balance_transaction_component,event_type,activity_at,activity_interval_type,activity_start_date,activity_end_date,currency,...,card_country,statement_descriptor,payment_metadata[destination],refund_id,dispute_id,transfer_id,connected_account_id,connected_account_name,connected_account_country,connected_account_direct_charge_id
904,txn_1NwevmFq1BiPgN2KvrGJ7jHi,2023-10-02 05:44:34,stripe_fees,other_fee,fee,2023-10-01 23:59:59,range,2023-10-01,2023-10-01,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5003,txn_1Nx1N5Fq1BiPgN2KeoChbBN9,2023-10-03 05:42:15,stripe_fees,other_fee,fee,2023-10-02 23:59:59,range,2023-10-02,2023-10-02,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8569,txn_1NxRIAFq1BiPgN2KPTpT1lmx,2023-10-04 09:22:54,stripe_fees,other_fee,fee,2023-10-03 23:59:59,range,2023-10-03,2023-10-03,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11366,txn_1NxkbDFq1BiPgN2Knv8PZa27,2023-10-05 05:59:51,stripe_fees,other_fee,fee,2023-10-04 23:59:59,range,2023-10-04,2023-10-04,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12102,txn_1Ny8RHFq1BiPgN2KwMBDCqZV,2023-10-06 07:27:11,stripe_fees,other_fee,fee,2023-10-05 23:59:59,range,2023-10-05,2023-10-05,eur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
list_account = df[['connected_account_id','connected_account_name']].groupby(['connected_account_id','connected_account_name']).mean()
list_account.to_csv('list_account.csv')

In [75]:
charge_per_m = df[['charge_id','connected_account_name']].groupby(['connected_account_name']).nunique()
charge_per_m

,charge_id
connected_account_name,
A Vos Conserves,4
ALL TIGERS,13
ART OF SOULE,39
AVLZ Marketing,30
Adorna,17
...,...
Yacon & Co,128
Yolaine,255
Ypson's,14
